In [1]:
import pandas as pd
import numpy as np
import random
import statistics
from sklearn.linear_model import LinearRegression
import time
from Metrics import Metrics


data = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Final_data.xlsx")
# Izberi nakjlučni vzorec velikosti 300

# Clean sample

data_test = data[data.Train_test_identificator == "test"]
data_train = data[data.Train_test_identificator == "train"]


data_train = data_train.reset_index()
data_test = data_test.reset_index()


group_data = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\Group_Data.xlsx")

decili_ccm_kw = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Final data\\decili_ccm_kw.xlsx")
decili_ccm = list(decili_ccm_kw["decili_ccm"].values)
decili_kw = list(decili_ccm_kw["decili_kw"].values)

def is_nan(n):
    if str(n) == "nan":
        return True
    return False


# Definiraj funkcije za primerjavo

def primerjava_tipa(avto_main_tip, avto_main_znamka, avto_comp_tip, avto_comp_znamka, alfa_tip):
    # Če sta enaka vrni alfa
    if avto_main_tip == avto_comp_tip and avto_main_znamka == avto_comp_znamka:
        return alfa_tip
    
    # Če nista poglej, če imata oba count večji od 3
    
    main_data = group_data[(group_data.tip == avto_main_tip) & (group_data.znamka == avto_main_znamka)]
    
    comp_data = group_data[(group_data.tip == avto_comp_tip) & (group_data.znamka == avto_comp_znamka)]
    
    if len(main_data.index) > 0 and len(comp_data.index) > 0:
        main_mean = main_data["mean"].values[0]
        comp_mean = comp_data["mean"].values[0]
        
        rel_diff = abs((comp_mean - main_mean) / main_mean)
        
        return (1-rel_diff) * alfa_tip
    
    return 0

def primerjava_starosti(main_prva_reg, comp_prva_reg, alfa_starost):
    if is_nan(main_prva_reg) == False and is_nan(comp_prva_reg) == False:
        diff = min(abs(main_prva_reg-comp_prva_reg),30)
        return (30 - diff) * alfa_starost
    return 0

def primerjava_ccm(main_ccm_class, comp_ccm_class, alfa_ccm):
    if is_nan(main_ccm_class) == False and is_nan(comp_ccm_class)== False: 
        diff = 10- abs(main_ccm_class - comp_ccm_class)
        return diff *alfa_ccm
    return 0

def primerjava_kw(main_kw_class, comp_kw_class, alfa_kw):
    if is_nan(main_kw_class) == False and is_nan(comp_kw_class)== False: 
        diff = 10- abs(main_kw_class - comp_kw_class)
        return diff *alfa_kw
    return 0

def primerjava_prevozeni_km(main_km, comp_km, alfa_km):
    if is_nan(main_km) == False and is_nan(comp_km)== False:
        diff = abs(main_km-comp_km)
        diff = min(diff, 300000)
        return alfa_km * (300000 - diff)
    return 0

def primerjava_goriva(main_gorivo, comp_gorivo, alfa_gorivo):
    if is_nan(main_gorivo) == False and is_nan(comp_gorivo)== False:
        if main_gorivo == comp_gorivo:
            return alfa_gorivo
    return 0

def primerjava_menjalnik(main_menjalnik, comp_menjalnik, alfa_menjalnik):
    if is_nan(main_menjalnik) == False and is_nan(comp_menjalnik)== False:
        if main_menjalnik == comp_menjalnik:
            return alfa_menjalnik
    return 0

def primerjava_oblika(main_oblika, comp_oblika, alfa_oblika):
    if is_nan(main_oblika) == False and is_nan(comp_oblika)== False:
        if main_oblika == comp_oblika:
            return alfa_oblika
    return 0
    
    
def primerjava_barva(main_barva, comp_barva, alfa_barva):
    if is_nan(main_barva) == False and is_nan(comp_barva)== False:
        if main_barva == comp_barva:
            return alfa_barva
    return 0


def Model_no_adjustments(train_data, test_data, parameters, number_of_neighbours = 5,run_name = None, save_name =None, save_results = False, save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\"):
    
    # Initialize parameters
    alfa_tip = parameters[0]
    alfa_starost = parameters[1]
    alfa_ccm = parameters[2]
    alfa_kw = parameters[3]
    alfa_km = parameters[4] # Rationale 100,000 km je prbližno enako 3 leta starosti
    alfa_gorivo = parameters[5]
    alfa_menjalnik = parameters[6]
    alfa_oblika = parameters[7]
    alfa_barva = parameters[8]
    
    # Define empty list to save data
    out = []
    
    # Merge with group data
    
    train_data = pd.merge(train_data, group_data, on=["tip", "znamka"], how = "left")
    test_data = pd.merge(test_data, group_data, on=["tip", "znamka"], how = "left")
    
    
    train_data = train_data.reset_index()
    test_data = test_data.reset_index()
    # Loop through all cars to be evaluated
    for i in range(0,len(test_data.index)):
    
        mere_podobnosti = []
        tip = []
        starost = []
        ccm = []
        kw = []
        km = []
        gorivo = []
        menjalnik = []
        oblika = []
        barva = []
        
        # Filter out cars that have average price +- 20 %
        filter_out_value = 1.5
        avg_price_main = test_data["mean"].values[i]
        
        try:
            avg_price_main = float(avg_price_main)
            train_data_for_specific_car = train_data[(train_data.cena < avg_price_main * (1+filter_out_value)) & (train_data.cena > avg_price_main * (1-filter_out_value))]
        except ValueError:
            print("Mean price of car not available")
            train_data_for_specific_car = pd.DataFrame(train_data)
         
        # Če smo obdržali dovolj veliko število avtomobilov, potem nadaljujemo, če ne glejmo vse avtomobile
        if len(train_data_for_specific_car.index) < 50:
            train_data_for_specific_car = pd.DataFrame(train_data)
            
        
        train_data_for_specific_car = train_data_for_specific_car.reset_index(drop=True)
        
        for j in range(0,len(train_data_for_specific_car.index)):

            podobnost = 0

            # Primerjava Tipa

            avto_main_tip = test_data["tip"].values[i]
            avto_main_znamka = test_data["znamka"].values[i]
            avto_comp_tip = train_data_for_specific_car["tip"].values[j]
            avto_comp_znamka = train_data_for_specific_car["znamka"].values[j]

            p = primerjava_tipa(avto_main_tip, avto_main_znamka, avto_comp_tip, avto_comp_znamka, alfa_tip)
            tip.append(p)
            podobnost += p

            # Primerjava starosti

            main_prva_reg =  test_data["leto_prve_registracije"].values[i]
            comp_prva_reg =  train_data_for_specific_car["leto_prve_registracije"].values[j]

            p =  primerjava_starosti(main_prva_reg, comp_prva_reg, alfa_starost)
            starost.append(p)
            podobnost += p     

            # Primerjava ccm

            main_ccm = test_data["ccm_class"].values[i]
            comp_ccm = train_data_for_specific_car["ccm_class"].values[j]

            p = primerjava_ccm(main_ccm, comp_ccm, alfa_ccm)
            ccm.append(p)
            podobnost += p

            # Primerjava kw

            main_kw =  test_data["kw_class"].values[i]
            comp_kw = train_data_for_specific_car["kw_class"].values[j]

            p = primerjava_kw(main_kw, comp_kw, alfa_kw)
            kw.append(p)
            podobnost += p

            # Prevozeni km

            main_km =  test_data["prevozeni_km"].values[i]
            comp_km = train_data_for_specific_car["prevozeni_km"].values[j]

            p = primerjava_prevozeni_km(main_km, comp_km, alfa_km)
            km.append(p)
            podobnost += p

            # Primerjava gorivo

            main_gorivo =  test_data["gorivo"].values[i]
            comp_gorivo = train_data_for_specific_car["gorivo"].values[j]

            p = primerjava_goriva(main_gorivo, comp_gorivo, alfa_gorivo)
            gorivo.append(p)
            podobnost += p


            # Menjalnik

            main_menjalnik = test_data["menjalnik"].values[i]
            comp_menjalnik =train_data_for_specific_car["menjalnik"].values[j]

            p = primerjava_menjalnik(main_menjalnik, comp_menjalnik, alfa_menjalnik)
            menjalnik.append(p)
            podobnost += p

            # Oblika

            main_oblika= test_data["oblika"].values[i]
            comp_oblika = train_data_for_specific_car["oblika"].values[j]

            p = primerjava_oblika(main_oblika, comp_oblika, alfa_oblika)
            oblika.append(p)
            podobnost += p

            # Barva

            main_barva = test_data["barva"].values[i]
            comp_barva = train_data_for_specific_car["barva"].values[j]


            p =  primerjava_barva(main_barva, comp_barva, alfa_barva)
            barva.append(p)
            podobnost += p

            # Konec primerjav

            mere_podobnosti.append(podobnost)

        train_data_for_specific_car["mere_podobnosti"] = mere_podobnosti
        train_data_for_specific_car["atribut_tip"] = tip 
        train_data_for_specific_car["atribut_starost"] =starost
        train_data_for_specific_car["atribut_ccm"] = ccm 
        train_data_for_specific_car["atribut_kw"] =kw 
        train_data_for_specific_car["atribut_prev_km"] =km 
        train_data_for_specific_car["atribut_gorivo"] =gorivo 
        train_data_for_specific_car["atribut_menjalnik"] =menjalnik 
        train_data_for_specific_car["atribut_oblika"] = oblika 
        train_data_for_specific_car["atribut_barva"] =barva 


        comparable_cars = train_data_for_specific_car.sort_values(by= "mere_podobnosti", ascending =False)

        comparable_cars_final_candidates = comparable_cars.head(number_of_neighbours)
        
        IDS_comp = comparable_cars_final_candidates["ID"].values
        valuation = comparable_cars_final_candidates["cena"].mean()
        ID_main = test_data["ID"].values[i]
        Cena_main =test_data["cena"].values[i]
        Mere_podbnosti_comp = comparable_cars_final_candidates["ID"].values
        
        if i % 10 == 0:
            print(i)
            
        dict_out = {"ID" : ID_main, "ocena" : valuation, "cena" : Cena_main, "IDS_of_top_k_matches" : IDS_comp,"Mere_podobnosti" :Mere_podbnosti_comp, "Parameters" : parameters, "K" : number_of_neighbours }
        out.append(dict_out)
        
    # Tukaj sedeaj shrani rezultate
    
    Model_results = pd.DataFrame(out)
    
    if save_results:
        output_directory = save_directory + save_name
        
        with pd.ExcelWriter(output_directory) as writer: 

            Model_results.to_excel(writer, sheet_name = "Model_results")
            
            Model_metrics = Metrics(Model_results,run_name ,"ocena",parameters)
            
            Model_metrics.to_excel(writer, sheet_name = "Metrics")

    return Model_results



In [7]:
alfa_tip = 125
alfa_starost = 3 
alfa_ccm = 5
alfa_kw = 5
alfa_km = 1 / 3000 # Rationale 100,000 km je prbližno enako 3 leta starosti
alfa_gorivo = 20
alfa_menjalnik = 15
alfa_oblika = 15
alfa_barva = 1

parametri = [alfa_tip,alfa_starost,alfa_ccm,alfa_kw,alfa_km,alfa_gorivo,alfa_menjalnik,alfa_oblika,alfa_barva]

Model_no_adjustments(data_train,
                     data_test,
                     parametri,
                     number_of_neighbours = 5,
                     run_name = "Model no adjustments k is 5", 
                     save_name  = "Model_no_adjustment_results.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,3705.4,3790,"[5872, 206, 669, 5406, 6383]","[5872, 206, 669, 5406, 6383]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",5
1,11,24835.8,19990,"[6262, 8008, 2819, 4261, 6122]","[6262, 8008, 2819, 4261, 6122]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",5
2,14,22108.0,24990,"[748, 179, 5839, 7537, 9995]","[748, 179, 5839, 7537, 9995]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",5
3,19,5496.6,6990,"[1562, 9162, 3839, 4163, 1282]","[1562, 9162, 3839, 4163, 1282]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",5
4,24,4435.4,3399,"[2302, 6724, 4532, 9934, 7881]","[2302, 6724, 4532, 9934, 7881]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",5
...,...,...,...,...,...,...,...
995,10170,23006.0,18490,"[7032, 6638, 5159, 10035, 9511]","[7032, 6638, 5159, 10035, 9511]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",5
996,10177,5337.8,4500,"[3543, 4123, 7365, 7906, 3210]","[3543, 4123, 7365, 7906, 3210]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",5
997,10178,8137.8,11990,"[4749, 1033, 8566, 795, 183]","[4749, 1033, 8566, 795, 183]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",5
998,10194,2286.0,6990,"[4807, 7659, 6223, 7478, 267]","[4807, 7659, 6223, 7478, 267]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",5


In [29]:
alfa_tip = 125
alfa_starost = 3 
alfa_ccm = 5
alfa_kw = 5
alfa_km = 1 / 3000 # Rationale 100,000 km je prbližno enako 3 leta starosti
alfa_gorivo = 20
alfa_menjalnik = 15
alfa_oblika = 15
alfa_barva = 1

parametri = [alfa_tip,alfa_starost,alfa_ccm,alfa_kw,alfa_km,alfa_gorivo,alfa_menjalnik,alfa_oblika,alfa_barva]

save = Model_no_adjustments(data_train, data_test, parametri, number_of_neighbours = 5)

save.to_excel("C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\Model_results_no_adjustment_k_is_5.xlsx")


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [62]:
from Model_no_adjustments import Model_no_adjustments

alfa_tip = 125
alfa_starost = 3 
alfa_ccm = 5
alfa_kw = 5
alfa_km = 1 / 3000 # Rationale 100,000 km je prbližno enako 3 leta starosti
alfa_gorivo = 20
alfa_menjalnik = 15
alfa_oblika = 15
alfa_barva = 1

parametri = [alfa_tip,alfa_starost,alfa_ccm,alfa_kw,alfa_km,alfa_gorivo,alfa_menjalnik,alfa_oblika,alfa_barva]

Model_no_adjustments(data_train,
                     data_test,
                     parametri,
                     number_of_neighbours = 5,
                     run_name = "Model no adjustments.xlsx", 
                     save_name  = "Model_no_adjustment_results.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


NameError: name 'parametri' is not defined

In [8]:
Model_no_adjustments(data_train,
                     data_test,
                     parametri,
                     number_of_neighbours = 3,
                     run_name = "Model no adjustments k = 3", 
                     save_name  = "Model_no_adjustment_results_k_is_3.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,3642.333333,3790,"[5872, 206, 669]","[5872, 206, 669]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",3
1,11,24093.000000,19990,"[6262, 8008, 2819]","[6262, 8008, 2819]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",3
2,14,23716.666667,24990,"[748, 179, 5839]","[748, 179, 5839]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",3
3,19,5013.333333,6990,"[1562, 9162, 3839]","[1562, 9162, 3839]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",3
4,24,4733.000000,3399,"[2302, 6724, 4532]","[2302, 6724, 4532]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",3
...,...,...,...,...,...,...,...
995,10170,22683.333333,18490,"[7032, 6638, 5159]","[7032, 6638, 5159]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",3
996,10177,5429.666667,4500,"[3543, 4123, 7365]","[3543, 4123, 7365]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",3
997,10178,7733.000000,11990,"[4749, 1033, 8566]","[4749, 1033, 8566]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",3
998,10194,2860.000000,6990,"[4807, 7659, 6223]","[4807, 7659, 6223]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",3


In [9]:
Model_no_adjustments(data_train,
                     data_test,
                     parametri,
                     number_of_neighbours = 4,
                     run_name = "Model no adjustments k = 4", 
                     save_name  = "Model_no_adjustment_results_k_is_4.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,3956.75,3790,"[5872, 206, 669, 5406]","[5872, 206, 669, 5406]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
1,11,24794.75,19990,"[6262, 8008, 2819, 4261]","[6262, 8008, 2819, 4261]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
2,14,22785.00,24990,"[748, 179, 5839, 7537]","[748, 179, 5839, 7537]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
3,19,5321.00,6990,"[1562, 9162, 3839, 4163]","[1562, 9162, 3839, 4163]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
4,24,5047.25,3399,"[2302, 6724, 4532, 9934]","[2302, 6724, 4532, 9934]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
...,...,...,...,...,...,...,...
995,10170,24760.00,18490,"[7032, 6638, 5159, 10035]","[7032, 6638, 5159, 10035]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
996,10177,5797.25,4500,"[3543, 4123, 7365, 7906]","[3543, 4123, 7365, 7906]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
997,10178,7797.25,11990,"[4749, 1033, 8566, 795]","[4749, 1033, 8566, 795]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
998,10194,2545.00,6990,"[4807, 7659, 6223, 7478]","[4807, 7659, 6223, 7478]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4


In [10]:
Model_no_adjustments(data_train,
                     data_test,
                     parametri,
                     number_of_neighbours = 6,
                     run_name = "Model no adjustments k = 6", 
                     save_name  = "Model_no_adjustment_results_k_is_6.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,3496.166667,3790,"[5872, 206, 669, 5406, 6383, 1303]","[5872, 206, 669, 5406, 6383, 1303]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",6
1,11,26992.666667,19990,"[6262, 8008, 2819, 4261, 6122, 1739]","[6262, 8008, 2819, 4261, 6122, 1739]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",6
2,14,22090.000000,24990,"[748, 179, 5839, 7537, 9995, 2178]","[748, 179, 5839, 7537, 9995, 2178]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",6
3,19,4880.500000,6990,"[1562, 9162, 3839, 4163, 1282, 9230]","[1562, 9162, 3839, 4163, 1282, 9230]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",6
4,24,4412.833333,3399,"[2302, 6724, 4532, 9934, 7881, 7943]","[2302, 6724, 4532, 9934, 7881, 7943]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",6
...,...,...,...,...,...,...,...
995,10170,21821.666667,18490,"[7032, 6638, 5159, 10035, 9511, 7858]","[7032, 6638, 5159, 10035, 9511, 7858]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",6
996,10177,5663.166667,4500,"[3543, 4123, 7365, 7906, 3210, 1308]","[3543, 4123, 7365, 7906, 3210, 1308]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",6
997,10178,8773.166667,11990,"[4749, 1033, 8566, 795, 183, 2088]","[4749, 1033, 8566, 795, 183, 2088]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",6
998,10194,2446.666667,6990,"[4807, 7659, 6223, 7478, 267, 848]","[4807, 7659, 6223, 7478, 267, 848]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",6


In [12]:
Model_no_adjustments(data_train,
                     data_test,
                     parametri,
                     number_of_neighbours = 7,
                     run_name = "Model no adjustments k = 7", 
                     save_name  = "Model_no_adjustment_results_k_is_7.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,3781.000000,3790,"[5872, 206, 669, 5406, 6383, 1303, 7002]","[5872, 206, 669, 5406, 6383, 1303, 7002]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",7
1,11,26208.000000,19990,"[6262, 8008, 2819, 4261, 6122, 1739, 6175]","[6262, 8008, 2819, 4261, 6122, 1739, 6175]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",7
2,14,20961.428571,24990,"[748, 179, 5839, 7537, 9995, 2178, 10]","[748, 179, 5839, 7537, 9995, 2178, 10]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",7
3,19,4953.285714,6990,"[1562, 9162, 3839, 4163, 1282, 9230, 8678]","[1562, 9162, 3839, 4163, 1282, 9230, 8678]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",7
4,24,4346.714286,3399,"[2302, 6724, 4532, 9934, 7881, 7943, 860]","[2302, 6724, 4532, 9934, 7881, 7943, 860]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",7
...,...,...,...,...,...,...,...
995,10170,22982.857143,18490,"[7032, 6638, 5159, 10035, 9511, 7858, 2976]","[7032, 6638, 5159, 10035, 9511, 7858, 2976]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",7
996,10177,5837.142857,4500,"[3543, 4123, 7365, 7906, 3210, 1308, 5970]","[3543, 4123, 7365, 7906, 3210, 1308, 5970]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",7
997,10178,9418.428571,11990,"[4749, 1033, 8566, 795, 183, 2088, 3608]","[4749, 1033, 8566, 795, 183, 2088, 3608]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",7
998,10194,2461.428571,6990,"[4807, 7659, 6223, 7478, 267, 848, 5749]","[4807, 7659, 6223, 7478, 267, 848, 5749]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",7


In [13]:
Model_no_adjustments(data_train,
                     data_test,
                     parametri,
                     number_of_neighbours = 4,
                     run_name = "Model no adjustments k = 4", 
                     save_name  = "Model_no_adjustment_results_k_is_4.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,3956.75,3790,"[5872, 206, 669, 5406]","[5872, 206, 669, 5406]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
1,11,24794.75,19990,"[6262, 8008, 2819, 4261]","[6262, 8008, 2819, 4261]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
2,14,22785.00,24990,"[748, 179, 5839, 7537]","[748, 179, 5839, 7537]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
3,19,5321.00,6990,"[1562, 9162, 3839, 4163]","[1562, 9162, 3839, 4163]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
4,24,5047.25,3399,"[2302, 6724, 4532, 9934]","[2302, 6724, 4532, 9934]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
...,...,...,...,...,...,...,...
995,10170,24760.00,18490,"[7032, 6638, 5159, 10035]","[7032, 6638, 5159, 10035]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
996,10177,5797.25,4500,"[3543, 4123, 7365, 7906]","[3543, 4123, 7365, 7906]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
997,10178,7797.25,11990,"[4749, 1033, 8566, 795]","[4749, 1033, 8566, 795]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4
998,10194,2545.00,6990,"[4807, 7659, 6223, 7478]","[4807, 7659, 6223, 7478]","[125, 3, 5, 5, 0.0003333333333333333, 20, 15, ...",4


In [16]:
# subsample IDs
import random

df_subsample_ids = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Other\\Ids_test_subsample.xlsx")
subsample_ids = list(df_subsample_ids["IDs_subsample"].values)

# Za vzorec velikosti 300 poizkusi nekaj naključnih parametrizacij

data_test_subsample = data_test[data_test['ID'].isin(subsample_ids)]

alfa_tip = 125
alfa_starost = 3 
alfa_ccm = 5
alfa_kw = 5
alfa_km = 1 / 3000 # Rationale 100,000 km je prbližno enako 3 leta starosti
alfa_gorivo = 20
alfa_menjalnik = 15
alfa_oblika = 15
alfa_barva = 1

parametri = [alfa_tip,alfa_starost,alfa_ccm,alfa_kw,alfa_km,alfa_gorivo,alfa_menjalnik,alfa_oblika,alfa_barva]


# Za 10 naključnih parametrizacij poskusi pognati na vzorcu 300 (ki je vedno enak), k = 6 konstanta


for i in range(0,10):
    print(f"This is run {i}")
    # Make copy of parametri
    parametri_copy = list(parametri)
    parametri_copy_out = []
    # Zapelji se čez parametre in s verjetnostjo 1/2 parameter povečaj za X porazdeljen U(1,10), z verjetnostjo 1/2 pa ga pomnoži z X-om porazdeljenim U(0,1)
    for p in parametri_copy:
        random_number = random.random()
        if random_number > 0.5:
            new_p = 1 + p * 9
            parametri_copy_out.append(new_p)
        else:
            new_p = p * random.random()
            parametri_copy_out.append(new_p)
            
    # Sedaj imaš "randomizirane parametre"
    
    print(parametri_copy_out)
    
    Model_no_adjustments(data_train,
                     data_test_subsample,
                     parametri_copy_out,
                     number_of_neighbours = 6,
                     run_name = "Model no adjustment randomized", 
                     save_name  = "Model_no_adjustment_k_is_5_randomized_run_" + str(i) + ".xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )

        
    
    

This is run 0
[110.43961007298466, 28, 46, 4.501065673493676, 0.0001535603957129805, 11.658829524850114, 9.697148254919576, 8.807071366193512, 10]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 1
[17.79336880152728, 1.9118848166093403, 46, 46, 0.0003171675060131229, 181, 5.985885498071959, 136, 10]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 2
[106.92799461725852, 28, 3.830465155169136, 3.899250090887052, 1.003, 5.540343772168623, 136, 3.4941136694542925, 10]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 3
[1126, 0.5860825887646063, 3.9483800771778994, 46, 6.321276719739773e-06, 11.812797468646789, 136, 136, 10]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 4
[1126, 28, 1.688117467214492, 4

In [18]:
import random

df_subsample_ids = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Other\\Ids_test_subsample.xlsx")
subsample_ids = list(df_subsample_ids["IDs_subsample"].values)

# Za vzorec velikosti 300 poizkusi nekaj naključnih parametrizacij

data_test_subsample = data_test[data_test['ID'].isin(subsample_ids)]

alfa_tip = 125
alfa_starost = 3 
alfa_ccm = 5
alfa_kw = 5
alfa_km = 1 / 3000 # Rationale 100,000 km je prbližno enako 3 leta starosti
alfa_gorivo = 20
alfa_menjalnik = 15
alfa_oblika = 15
alfa_barva = 1

parametri = [alfa_tip,
             alfa_starost,
             alfa_ccm,
             alfa_kw,
             alfa_km,
             alfa_gorivo,
             alfa_menjalnik,
             alfa_oblika,
             alfa_barva]
parametri = [63.22345509493234, 28, 46, 46, 0.00020863861850860035, 181, 136, 5.054321049593293, 0.7242649646450485]

# Za 10 naključnih parametrizacij poskusi pognati na vzorcu 300 (ki je vedno enak), k = 6 konstanta


for i in range(30,32):
    print(f"This is run {i}")
    # Make copy of parametri
    parametri_copy = list(parametri)
    parametri_copy_out = []
    # Zapelji se čez parametre in s verjetnostjo 1/2 parameter povečaj za X porazdeljen U(1,10), z verjetnostjo 1/2 pa ga pomnoži z X-om porazdeljenim U(0,1)
    for p in parametri_copy:
        random_number = random.random()
        if random_number > 0.5:
            new_p = 1 + p * 9
            parametri_copy_out.append(new_p)
        else:
            new_p = p * random.random()
            parametri_copy_out.append(new_p)
            
    # Sedaj imaš "randomizirane parametre"
    
    print(parametri_copy_out)
    
    Model_no_adjustments(data_train,
                     data_test_subsample,
                     parametri_copy_out,
                     number_of_neighbours = 6,
                     run_name = "Model no adjustment randomized", 
                     save_name  = "Model_no_adjustment_k_is_5_randomized_run_" + str(i) + ".xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )

        
    
    

This is run 30
[38.141433666478136, 28, 46, 46, 1.003, 181, 9.57340176191183, 6.37144416148981, 0.9094985052361415]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 31
[1126, 28, 0.6223366359369237, 0.03313413107505159, 0.00017137809127732497, 181, 11.59057225671117, 136, 0.1374557577673472]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290


In [ ]:
Model_no_adjustments(data_train,
                 data_test_subsample,
                 parametri_copy_out,
                 number_of_neighbours = 6,
                 run_name = "Model no adjustment randomized", 
                 save_name  = "Model_no_adjustment_k_is_5_randomized_run_" + str(i) + ".xlsx",
                 save_results = True,
                 save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )



    

In [24]:
parametri = [alfa_tip* 0.5,
             alfa_starost * 3,
             alfa_ccm * 2,
             alfa_kw*2,
             alfa_km * 2,
             alfa_gorivo,
             alfa_menjalnik,
             alfa_oblika,
             alfa_barva]
Model_no_adjustments(data_train,
                     data_test_subsample,
                     parametri_copy_out,
                     number_of_neighbours = 6,
                     run_name = "Model no adjustment randomized", 
                     save_name  = "Model_no_adjustment_k_is_5_randomized_run_" + str(i) + ".xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )

        
    

0


KeyboardInterrupt: 

In [27]:
parametri = [alfa_tip* 0.5,
             alfa_starost * 3,
             alfa_ccm * 2,
             alfa_kw*2,
             alfa_km * 2,
             alfa_gorivo,
             alfa_menjalnik,
             alfa_oblika,
             alfa_barva]
Model_no_adjustments(data_train,
                     data_test_subsample,
                     parametri,
                     number_of_neighbours = 6,
                     run_name = "Model no adjustment randomized", 
                     save_name  = "Model_no_adjustment_guess" + str(i) + ".xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )

        

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290


,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,3666.666667,3790,"[5406, 5872, 1303, 6383, 206, 7473]","[5406, 5872, 1303, 6383, 206, 7473]","[62.5, 9, 10, 10, 0.0006666666666666666, 20, 1...",6
1,14,22500.000000,24990,"[748, 179, 7537, 5839, 9142, 5414]","[748, 179, 7537, 5839, 9142, 5414]","[62.5, 9, 10, 10, 0.0006666666666666666, 20, 1...",6
2,19,5455.500000,6990,"[1562, 9162, 3839, 4163, 1282, 8827]","[1562, 9162, 3839, 4163, 1282, 8827]","[62.5, 9, 10, 10, 0.0006666666666666666, 20, 1...",6
3,51,18531.666667,20750,"[3446, 1225, 2755, 8500, 424, 1728]","[3446, 1225, 2755, 8500, 424, 1728]","[62.5, 9, 10, 10, 0.0006666666666666666, 20, 1...",6
4,69,2955.000000,2490,"[834, 4613, 1560, 7172, 5440, 498]","[834, 4613, 1560, 7172, 5440, 498]","[62.5, 9, 10, 10, 0.0006666666666666666, 20, 1...",6
...,...,...,...,...,...,...,...
295,9983,20236.666667,19890,"[8536, 6404, 9236, 9912, 1763, 453]","[8536, 6404, 9236, 9912, 1763, 453]","[62.5, 9, 10, 10, 0.0006666666666666666, 20, 1...",6
296,10074,16711.666667,19950,"[7330, 4313, 1425, 802, 7034, 6996]","[7330, 4313, 1425, 802, 7034, 6996]","[62.5, 9, 10, 10, 0.0006666666666666666, 20, 1...",6
297,10084,4856.500000,2290,"[1819, 5033, 6460, 5773, 4606, 6354]","[1819, 5033, 6460, 5773, 4606, 6354]","[62.5, 9, 10, 10, 0.0006666666666666666, 20, 1...",6
298,10158,3298.333333,3350,"[6784, 1472, 728, 2185, 8201, 8184]","[6784, 1472, 728, 2185, 8201, 8184]","[62.5, 9, 10, 10, 0.0006666666666666666, 20, 1...",6


In [28]:
parametri = [alfa_tip* 0.6,
             alfa_starost * 3,
             alfa_ccm * 3,
             alfa_kw,
             alfa_km * 3,
             alfa_gorivo,
             alfa_menjalnik,
             alfa_oblika,
             alfa_barva]
Model_no_adjustments(data_train,
                     data_test_subsample,
                     parametri,
                     number_of_neighbours = 6,
                     run_name = "Model no adjustment randomized", 
                     save_name  = "Model_no_adjustment_guess2" + str(i) + ".xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )

        
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290


,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,3448.333333,3790,"[5406, 5872, 1303, 6383, 206, 9427]","[5406, 5872, 1303, 6383, 206, 9427]","[75.0, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
1,14,22500.833333,24990,"[748, 179, 5414, 9142, 3803, 5839]","[748, 179, 5414, 9142, 3803, 5839]","[75.0, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
2,19,4880.500000,6990,"[1562, 9162, 3839, 4163, 1282, 9230]","[1562, 9162, 3839, 4163, 1282, 9230]","[75.0, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
3,51,18440.000000,20750,"[1225, 3446, 10184, 424, 8500, 2755]","[1225, 3446, 10184, 424, 8500, 2755]","[75.0, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
4,69,3238.333333,2490,"[834, 498, 4668, 1560, 7172, 4613]","[834, 498, 4668, 1560, 7172, 4613]","[75.0, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
...,...,...,...,...,...,...,...
295,9983,20236.666667,19890,"[6404, 8536, 9236, 9912, 1763, 453]","[6404, 8536, 9236, 9912, 1763, 453]","[75.0, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
296,10074,15213.333333,19950,"[7330, 4313, 1425, 802, 5584, 7034]","[7330, 4313, 1425, 802, 5584, 7034]","[75.0, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
297,10084,4349.833333,2290,"[1819, 5033, 5773, 6460, 6354, 7336]","[1819, 5033, 5773, 6460, 6354, 7336]","[75.0, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
298,10158,3298.333333,3350,"[6784, 2185, 1472, 8184, 728, 8201]","[6784, 2185, 1472, 8184, 728, 8201]","[75.0, 9, 15, 5, 0.001, 20, 15, 15, 1]",6


In [31]:
paramettri1 = [75.0 * 0.75, 9, 15, 5, 0.001, 20, 15, 15, 1]

Model_no_adjustments(data_train,
                     data_test_subsample,
                     paramettri1,
                     number_of_neighbours = 6,
                     run_name = "Model no adjustment randomized", 
                     save_name  = "Model_guessnew1.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )

        
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290


,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,3116.666667,3790,"[5406, 1303, 5872, 6383, 206, 8092]","[5406, 1303, 5872, 6383, 206, 8092]","[56.25, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
1,14,18620.833333,24990,"[748, 179, 5414, 6563, 1636, 3803]","[748, 179, 5414, 6563, 1636, 3803]","[56.25, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
2,19,4880.500000,6990,"[1562, 9162, 3839, 4163, 1282, 9230]","[1562, 9162, 3839, 4163, 1282, 9230]","[56.25, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
3,51,18440.000000,20750,"[1225, 3446, 10184, 424, 8500, 2755]","[1225, 3446, 10184, 424, 8500, 2755]","[56.25, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
4,69,2888.333333,2490,"[834, 498, 4668, 1560, 5440, 7172]","[834, 498, 4668, 1560, 5440, 7172]","[56.25, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
...,...,...,...,...,...,...,...
295,9983,20236.666667,19890,"[9236, 9912, 6404, 8536, 1763, 453]","[9236, 9912, 6404, 8536, 1763, 453]","[56.25, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
296,10074,15213.333333,19950,"[7330, 4313, 1425, 802, 1339, 5584]","[7330, 4313, 1425, 802, 1339, 5584]","[56.25, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
297,10084,4349.833333,2290,"[1819, 5033, 6460, 5773, 6354, 7336]","[1819, 5033, 6460, 5773, 6354, 7336]","[56.25, 9, 15, 5, 0.001, 20, 15, 15, 1]",6
298,10158,3298.333333,3350,"[6784, 1472, 2185, 728, 8184, 8201]","[6784, 1472, 2185, 728, 8184, 8201]","[56.25, 9, 15, 5, 0.001, 20, 15, 15, 1]",6


In [33]:


import random

df_subsample_ids = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Other\\Ids_test_subsample.xlsx")
subsample_ids = list(df_subsample_ids["IDs_subsample"].values)

# Za vzorec velikosti 300 poizkusi nekaj naključnih parametrizacij

data_test_subsample = data_test[data_test['ID'].isin(subsample_ids)]

alfa_tip = 125
alfa_starost = 3 
alfa_ccm = 5
alfa_kw = 5
alfa_km = 1 / 3000 # Rationale 100,000 km je prbližno enako 3 leta starosti
alfa_gorivo = 20
alfa_menjalnik = 15
alfa_oblika = 15
alfa_barva = 1

parametri = [75.0 , 9, 15, 5, 0.001, 20, 15, 15, 1]
# Za 10 naključnih parametrizacij poskusi pognati na vzorcu 300 (ki je vedno enak), k = 6 konstanta


for i in range(11,17):
    print(f"This is run {i}")
    # Make copy of parametri
    parametri_copy = list(parametri)
    parametri_copy_out = []
    # Zapelji se čez parametre in s verjetnostjo 1/2 parameter povečaj za X porazdeljen U(1,10), z verjetnostjo 1/2 pa ga pomnoži z X-om porazdeljenim U(0,1)
    for p in parametri_copy:
        random_number = random.random()
        if random_number > 0.5:
            new_p = 1 + p * 9
            parametri_copy_out.append(new_p)
        else:
            new_p = p * random.random()
            parametri_copy_out.append(new_p)
            
    # Sedaj imaš "randomizirane parametre"
    
    print(parametri_copy_out)
    
    Model_no_adjustments(data_train,
                     data_test_subsample,
                     parametri_copy_out,
                     number_of_neighbours = 6,
                     run_name = "Model no adjustment randomized", 
                     save_name  = "Model_no_adjucated_guess_new_parameters_" + str(i) + ".xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


This is run 11
[676.0, 0.22724634602406113, 136, 4.591763684147051, 0.0008882428617403682, 1.930531607257353, 136, 11.561566641598468, 0.9524503529857424]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 12
[676.0, 82, 136, 1.5823765457726209, 1.009, 17.31260753841785, 2.930455232551096, 136, 0.7710728987158944]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 13
[9.651373386099, 82, 136, 2.055150141582964, 1.009, 8.405350656566956, 136, 136, 0.486741139909317]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 14
[20.595909455641376, 8.12432151529575, 14.440290673950013, 46, 0.0006924464130142422, 14.68114573576067, 10.913449030942685, 10.59296555821578, 0.19227017357006293]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270


In [36]:
par1 = [600, 82, 0.46436464359728924, 46, 0.00017612087355251915, 11.503684461271302, 14.746369393346153, 136, 0.17705236744464192]
par2 = [500, 82, 0.46436464359728924, 46, 0.00017612087355251915, 11.503684461271302, 14.746369393346153, 136, 0.17705236744464192]
par3 = [400, 82, 0.46436464359728924, 46, 0.00017612087355251915, 11.503684461271302, 14.746369393346153, 136, 0.17705236744464192]
par4 = [300, 82, 0.46436464359728924, 46, 0.00017612087355251915, 11.503684461271302, 14.746369393346153, 136, 0.17705236744464192]

param =[par1,par2,par3,par4]



for par in param:  
    print(par)
    Model_no_adjustments(data_train,
                     data_test_subsample,
                     par,
                     number_of_neighbours = 6,
                     run_name = "Model no adjustment randomized", 
                     save_name  = "Model_no_adjucated_guess_new_parameters_" + str(par[0]) + ".xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )

    


[600, 82, 0.46436464359728924, 46, 0.00017612087355251915, 11.503684461271302, 14.746369393346153, 136, 0.17705236744464192]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
[500, 82, 0.46436464359728924, 46, 0.00017612087355251915, 11.503684461271302, 14.746369393346153, 136, 0.17705236744464192]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
[400, 82, 0.46436464359728924, 46, 0.00017612087355251915, 11.503684461271302, 14.746369393346153, 136, 0.17705236744464192]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
[300, 82, 0.46436464359728924, 46, 0.00017612087355251915, 11.503684461271302, 14.746369393346153, 136, 0.17705236744464192]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290


In [37]:

import random

df_subsample_ids = pd.read_excel("C:\\Users\\1roks\\Documents\\FMF\Matematika z računalnikom\\Project\\Data\\Other\\Ids_test_subsample.xlsx")
subsample_ids = list(df_subsample_ids["IDs_subsample"].values)

# Za vzorec velikosti 300 poizkusi nekaj naključnih parametrizacij

data_test_subsample = data_test[data_test['ID'].isin(subsample_ids)]

alfa_tip = 125
alfa_starost = 3 
alfa_ccm = 5
alfa_kw = 5
alfa_km = 1 / 3000 # Rationale 100,000 km je prbližno enako 3 leta starosti
alfa_gorivo = 20
alfa_menjalnik = 15
alfa_oblika = 15
alfa_barva = 1

parametri = [676.0, 82, 12.221962511850087, 46, 0.0006077044479933562, 7.0816733670198335, 4.782236264991494, 11.853378581172926, 0.6096676242550324]

# Za 10 naključnih parametrizacij poskusi pognati na vzorcu 300 (ki je vedno enak), k = 6 konstanta


for i in range(1,41):
    print(f"This is run {i}")
    # Make copy of parametri
    parametri_copy = list(parametri)
    parametri_copy_out = []
    # Zapelji se čez parametre in s verjetnostjo 1/2 parameter povečaj za X porazdeljen U(1,10), z verjetnostjo 1/2 pa ga pomnoži z X-om porazdeljenim U(0,1)
    for p in parametri_copy:
        random_number = random.random()
        if random_number > 0.5:
            new_p = 1 + p * 2
            parametri_copy_out.append(new_p)
        else:
            new_p = p * (0.5 + 0.5* random.random())
            parametri_copy_out.append(new_p)
            
    # Sedaj imaš "randomizirane parametre"
    
    print(parametri_copy_out)
    
    Model_no_adjustments(data_train,
                     data_test_subsample,
                     parametri_copy_out,
                     number_of_neighbours = 6,
                     run_name = "New x0", 
                     save_name  = "New_x0_" + str(i) + ".xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


This is run 1
[1353.0, 165, 25.443925023700174, 93, 1.0012154088959868, 15.163346734039667, 10.564472529982988, 7.41222507565992, 2.219335248510065]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 2
[639.7995267648562, 75.10027479700257, 6.913056738060228, 36.173693997366584, 1.0012154088959868, 4.522228951056359, 3.7974703247195767, 7.22741081657375, 0.543698079439953]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 3
[490.47420966617284, 165, 6.428830533774409, 43.57728359567411, 1.0012154088959868, 15.163346734039667, 3.7594608282767146, 24.706757162345852, 0.38643431986103594]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 4
[1353.0, 165, 25.443925023700174, 93, 0.0005897481049777209, 15.163346734039667, 10.564472529982988, 6.5949902141354455, 0.5191736241054278]
0


270
280
290
This is run 30
[1353.0, 165, 25.443925023700174, 26.729067986012552, 0.0006064276521505577, 15.163346734039667, 10.564472529982988, 7.224389144450007, 0.3985149437470053]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 31
[1353.0, 165, 25.443925023700174, 93, 0.0004644378895302149, 15.163346734039667, 2.4919193692709483, 24.706757162345852, 2.219335248510065]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 32
[481.1199854621589, 165, 25.443925023700174, 36.26787007580652, 1.0012154088959868, 15.163346734039667, 4.258974827433907, 10.95269902819117, 0.3731861103794081]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
This is run 33
[635.3041071836615, 70.06084428015537, 25.443925023700174, 25.59814075281484, 1.0012154088959868, 15.163346734039667, 10.564472529982988, 24.70

In [40]:
kandidat1 = [505.6615616440978, 75.85912730916039, 25.443925023700174, 93, 0.0003211615128590073, 6.336276868286331, 10.564472529982988, 10.75221198714343, 2.219335248510065]
kandidat2 = [466.08948962340173, 66.31282221516274, 8.032452666952638, 24.750775012632552, 0.0003428115825366971, 15.163346734039667, 10.564472529982988, 24.706757162345852, 0.5105554347604646]
kandidat3 = [607.9430923977272, 165, 11.775221465284474, 93, 0.0004937655384757594, 15.163346734039667, 10.564472529982988, 24.706757162345852, 0.5163549827531622]

Model_no_adjustments(data_train,
                     data_test,
                     kandidat1,
                     number_of_neighbours = 6,
                     run_name = "kandidat 1", 
                     save_name  = "kandidat 1.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )

Model_no_adjustments(data_train,
                     data_test,
                     kandidat2,
                     number_of_neighbours = 6,
                     run_name = "kandidat 2", 
                     save_name  = "kandidat 2.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )

Model_no_adjustments(data_train,
                     data_test,
                     kandidat3,
                     number_of_neighbours = 6,
                     run_name = "kandidat 3", 
                     save_name  = "kandidat 3.xlsx",
                     save_results = True,
                     save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
5

,ID,ocena,cena,IDS_of_top_k_matches,Mere_podobnosti,Parameters,K
0,9,5923.333333,3790,"[5406, 7002, 664, 5370, 7473, 4528]","[5406, 7002, 664, 5370, 7473, 4528]","[607.9430923977272, 165, 11.775221465284474, 9...",6
1,11,19948.166667,19990,"[6122, 6175, 1217, 7743, 2718, 1901]","[6122, 6175, 1217, 7743, 2718, 1901]","[607.9430923977272, 165, 11.775221465284474, 9...",6
2,14,19430.000000,24990,"[9142, 7537, 5884, 7431, 9531, 3677]","[9142, 7537, 5884, 7431, 9531, 3677]","[607.9430923977272, 165, 11.775221465284474, 9...",6
3,19,5862.166667,6990,"[1562, 9162, 4163, 1282, 9155, 9106]","[1562, 9162, 4163, 1282, 9155, 9106]","[607.9430923977272, 165, 11.775221465284474, 9...",6
4,24,3685.833333,3399,"[8757, 4532, 1735, 5275, 4309, 1814]","[8757, 4532, 1735, 5275, 4309, 1814]","[607.9430923977272, 165, 11.775221465284474, 9...",6
...,...,...,...,...,...,...,...
995,10170,20464.500000,18490,"[7032, 9682, 10110, 8611, 8758, 8032]","[7032, 9682, 10110, 8611, 8758, 8032]","[607.9430923977272, 165, 11.775221465284474, 9...",6
996,10177,4758.333333,4500,"[7943, 5797, 6724, 7937, 461, 8527]","[7943, 5797, 6724, 7937, 461, 8527]","[607.9430923977272, 165, 11.775221465284474, 9...",6
997,10178,7466.500000,11990,"[4749, 640, 135, 9457, 8566, 8707]","[4749, 640, 135, 9457, 8566, 8707]","[607.9430923977272, 165, 11.775221465284474, 9...",6
998,10194,3496.500000,6990,"[5749, 1996, 649, 6223, 848, 4807]","[5749, 1996, 649, 6223, 848, 4807]","[607.9430923977272, 165, 11.775221465284474, 9...",6


In [4]:
# Tukaj so še zadnji kandidati za katere je potrebno oceniti napovedno moč 

kandidati = [
[1353.0, 165, 25.443925023700174, 26.729067986012552, 0.0006064276521505577, 15.163346734039667, 10.564472529982988, 7.224389144450007, 0.3985149437470053],	
[1353.0, 165, 25.443925023700174, 28.869400273383537, 0.00047503438841323, 15.163346734039667, 3.048884217757374, 9.754385111457465, 2.219335248510065],
[1353.0, 165, 25.443925023700174, 93, 0.0004644378895302149, 15.163346734039667, 2.4919193692709483, 24.706757162345852, 2.219335248510065],
[1353.0, 165, 25.443925023700174, 93, 0.0005897481049777209, 15.163346734039667, 10.564472529982988, 6.5949902141354455, 0.5191736241054278],
[1353.0, 165, 25.443925023700174, 93, 0.0005927323031062219, 5.321620638504272, 2.448283446405696, 24.706757162345852, 2.219335248510065],
[1353.0, 44.658908117606586, 10.305766740774333, 93, 0.0004621984612858268, 6.837918863935737, 10.564472529982988, 24.706757162345852, 2.219335248510065],
[1353.0, 45.18627607683452, 25.443925023700174, 32.62349886536523, 0.00044316688447093827, 15.163346734039667, 10.564472529982988, 8.379269139045094, 0.39846168031766555],	
[1353.0, 71.5522204609932, 25.443925023700174, 37.84583892736889, 0.00041414461653806054, 6.004129520717076, 3.914669902089924, 9.33399952172518, 2.219335248510065],
[1353.0, 72.0483959474558, 11.921172160919927, 93, 0.0003927649976084962, 4.641244228979135, 10.564472529982988, 24.706757162345852, 0.5361470565650153],
[1353.0, 72.53066685508905, 7.240587962686981, 39.64950442503587, 0.00036798627458885263, 5.283113241277206, 2.724337183893586, 24.706757162345852, 2.219335248510065],
[1353.0, 80.19120243622852, 6.69370026005715, 24.02192642415504, 0.0003729608083909991, 15.163346734039667, 10.564472529982988, 8.490761706756397, 2.219335248510065],
[1353.0, 80.19120243622852, 6.69370026005715, 24.02192642415504, 0.0003729608083909991, 15.163346734039667, 10.564472529982988, 8.490761706756397, 2.219335248510065],
[390.94589727363876, 165, 25.443925023700174, 24.029700880042732, 0.0005666687604729718, 5.178282292352963, 10.564472529982988, 24.706757162345852, 0.5696183616721973],
[451.4323072892471, 62.89523654038864, 25.443925023700174, 93, 0.000584674244143278, 6.780451767207385, 3.6971154801464894, 11.066108390398307, 0.49409811576118196],
[466.08948962340173, 66.31282221516274, 8.032452666952638, 24.750775012632552, 0.0003428115825366971, 15.163346734039667, 10.564472529982988, 24.706757162345852, 0.5105554347604646],
[505.6615616440978, 75.85912730916039, 25.443925023700174, 93, 0.0003211615128590073, 6.336276868286331, 10.564472529982988, 10.75221198714343, 2.219335248510065],
[506.9721980741832, 165, 25.443925023700174, 34.50173085770483, 0.0005998981609443688, 3.7836276177344508, 4.360723888155504, 24.706757162345852, 0.4338995378444419],
[607.9430923977272, 165, 11.775221465284474, 93, 0.0004937655384757594, 15.163346734039667, 10.564472529982988, 24.706757162345852, 0.5163549827531622],
[657.8802136873337, 77.958497358364, 25.443925023700174, 93, 0.00042100987430586216, 5.373124388451237, 10.564472529982988, 10.526229141989761, 0.5551659968063513]]

# Zaokroži parametre
kandidati = [[round(j) for j in k] for k in kandidati]
kandidati = kandidati[4:]
stevec = 0
for parametri in kandidati:
    print(f"Tole je globalni stevec : {stevec}")
    stevec +=1
    st += 1
    for knn in range(5,7):
        Model_no_adjustments(data_train,
                         data_test,
                         parametri,
                         number_of_neighbours = knn,
                         run_name = "kandidat_" + str(st) + "knn = " + str(knn), 
                         save_name  = "kandidat_" + str(st) + "knn = " + str(knn) + ".xlsx",
                         save_results = True,
                         save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


Tole je globalni stevec : 0
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
Tole je globalni stevec : 1
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
4

290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
Tole je globalni stevec : 11
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
7

In [11]:
parametri = [[505.6615616440978, 75.85912730916039, 25.443925023700174, 93, 0.0003211615128590073, 6.336276868286331, 10.564472529982988, 10.75221198714343, 2.219335248510065],
[466.08948962340173, 66.31282221516274, 8.032452666952638, 24.750775012632552, 0.0003428115825366971, 15.163346734039667, 10.564472529982988, 24.706757162345852, 0.5105554347604646],
[1353.0, 44.658908117606586, 10.305766740774333, 93, 0.0004621984612858268, 6.837918863935737, 10.564472529982988, 24.706757162345852, 2.219335248510065],
[1353.0, 165, 25.443925023700174, 93, 0.0005927323031062219, 5.321620638504272, 2.448283446405696, 24.706757162345852, 2.219335248510065],
[1353.0, 45.18627607683452, 25.443925023700174, 32.62349886536523, 0.00044316688447093827, 15.163346734039667, 10.564472529982988, 8.379269139045094, 0.39846168031766555],
[1353.0, 72.53066685508905, 7.240587962686981, 39.64950442503587, 0.00036798627458885263, 5.283113241277206, 2.724337183893586, 24.706757162345852, 2.219335248510065],
[362.89578407753044, 81.55887312845084, 8.494455028555016, 93, 0.00032284026384319606, 15.163346734039667, 4.1353883592384575, 24.706757162345852, 2.219335248510065],
[451.4323072892471, 62.89523654038864, 25.443925023700174, 93, 0.000584674244143278, 6.780451767207385, 3.6971154801464894, 11.066108390398307, 0.49409811576118196],
[1353.0, 71.5522204609932, 25.443925023700174, 37.84583892736889, 0.00041414461653806054, 6.004129520717076, 3.914669902089924, 9.33399952172518, 2.219335248510065],
[607.9430923977272, 165, 11.775221465284474, 93, 0.0004937655384757594, 15.163346734039667, 10.564472529982988, 24.706757162345852, 0.5163549827531622],
[1353.0, 80.19120243622852, 6.69370026005715, 24.02192642415504, 0.0003729608083909991, 15.163346734039667, 10.564472529982988, 8.490761706756397, 2.219335248510065],
[390.94589727363876, 165, 25.443925023700174, 24.029700880042732, 0.0005666687604729718, 5.178282292352963, 10.564472529982988, 24.706757162345852, 0.5696183616721973],
[506.9721980741832, 165, 25.443925023700174, 34.50173085770483, 0.0005998981609443688, 3.7836276177344508, 4.360723888155504, 24.706757162345852, 0.4338995378444419]]


st = 1
p = [[round(j,6) for j in k] for k in parametri]
for parametri in p:
    print(f"Tole je globalni stevec : {stevec}")
    print(parametri)
    stevec +=1
    st += 1
    for knn in range(6,7):
        Model_no_adjustments(data_train,
                         data_test,
                         parametri,
                         number_of_neighbours = knn,
                         run_name = "Kandidat__" + str(st) + "knn = " + str(knn), 
                         save_name  = "Kandidat__" + str(st) + "knn = " + str(knn) + ".xlsx",
                         save_results = True,
                         save_directory="C:\\Users\\1roks\\Documents\\FMF\\Matematika z računalnikom\\Project\\Data\\Modeling results\\" )


Tole je globalni stevec : 18
[505.661562, 75.859127, 25.443925, 93, 0.000321, 6.336277, 10.564473, 10.752212, 2.219335]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
Tole je globalni stevec : 19
[466.08949, 66.312822, 8.032453, 24.750775, 0.000343, 15.163347, 10.564473, 24.706757, 0.510555]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
